In [1]:
import logging
logger = logging.getLogger()
for handler in logger.handlers:
    handler.level = logging.WARNING
handler = logging.FileHandler(filename='log.log', mode='w')
handler.level = logging.DEBUG
logger.level = logging.DEBUG
logger.addHandler(handler)

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
#import numpy as np
from autograd import numpy as np
import autograd
import sympy as sp
import bayesian_pdes as bpdes

In [3]:
from tempfile import NamedTemporaryFile

VIDEO_TAG = """<video autoplay loop controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)
from matplotlib import animation
animation.Animation._repr_html_ = anim_to_html

In [4]:
ls_t = 0.2
def k_t(t, tprime):
    return np.exp(-(t-tprime)**2 / (2*ls_t**2))
dk_t = autograd.grad(k_t)
d2k_t = autograd.grad(dk_t, 1)

In [5]:
x, y, t, tbar, alpha, theta, ls_x, ls_t, dt = sp.symbols('x,y,t,tbar,alpha,theta,ls_x,ls_t,dt')
def L_explicit(k):
    return k + alpha*theta*dt*k.diff(x,x)
def L_bar_explicit(k):
    return k + alpha*theta*dt*k.diff(y,y)
def L_implicit(k):
    return k - alpha*(1-theta)*dt*k.diff(x,x)
def L_bar_implicit(k):
    return k - alpha*(1-theta)*dt*k.diff(y,y)
def time_op(k):
    return alpha*k.diff(x,x)
def time_op_bar(k):
    return alpha*k.diff(y,y)
def k_t(k):
    return k.diff(t)
def k_tbar(k):
    return k.diff(tbar)
def B(k):
    return k
def Bbar(k):
    return k
def neumann(k):
    return k.diff(x)
def neumann_bar(k):
    return k.diff(y)

k = sp.exp(-(x-y)**2 / (2.*ls_x**2))*sp.exp(-(t-tbar)**2 / (2.*ls_t**2))

op_system = bpdes.operator_compilation.sympy_gram.compile_sympy(
    [L_implicit, L_explicit, k_t, B, neumann],
    [L_bar_implicit, L_bar_explicit, k_tbar, Bbar, neumann_bar],
    k,
    [[x, t], [y, tbar], [alpha, ls_x, ls_t, dt, theta]]
)

In [6]:
base_op_system = bpdes.operator_compilation.sympy_gram.compile_sympy(
    [time_op, neumann],
    [time_op_bar, neumann_bar],
    k,
    [[x, t], [y, tbar], [alpha, ls_x, ls_t]]
)

In [7]:
ops, ops_bar = [L_implicit, neumann], [L_bar_implicit, neumann_bar]

In [8]:
dt = 0.001
Tend = 0.4
dx = 0.01
Len = 1.
x_pts = np.arange(0,Len+dx,dx)[:,None]
x_int = x_pts[1:-1]
x_bdy = x_pts[[0, -1]]
times = np.arange(0,Tend+dt,dt)

In [9]:
def u0(x_pts):
    #return np.exp(np.random.normal(np.log(0.5), 0.1, x_pts.shape))
    #return 1-4*(x_pts-0.5)**2
    return 1./(1+np.exp(-20*(x_pts-0.5)))

In [10]:
alpha_val = 1.0
length_scale = 2*dx
base_args = np.array([alpha_val, length_scale, 0.1])

In [11]:
def augment_with_time(vec, t):
    time_component = t*np.ones((vec.shape[0], 1))
    return np.column_stack([vec, time_component])

In [12]:
np.random.seed(0)
ics = [(augment_with_time(x_pts, times[0]), u0(x_pts))]

So how should this code actually work? I need to be able to supply an operator which represents the time flow, and a collection of other operators. Then the thing needs to be able to retrieve the observations at time t along with any other observations and push the measure through time.

so the interface should look something like

```python
def solve_theta(time_ops, stationary_ops, op_system, ics, obs_function, times, theta, fun_args)
```

`time_ops` a list of lists, as is `stationary_ops`. `ics` tells us the initial conditions.

Then what I'm missing is a way to specify the forcing for the time operator. And the design points for the time operator. However I think these can both come through the obs function.

In [13]:
def solve_theta(ops, ops_bar, initial_obs, times, theta, other_args, debug=False):
    # assumption: times equally spaced
    dt = times[1]-times[0]
    
    # convention: last args are [dt, theta]
    fun_args = np.append(other_args, [dt, theta])
    print fun_args
    
    extra_initial_obs = [(augment_with_time(x_bdy, times[0]), np.array([[0.], [0.]]))]
    initial_posterior = bpdes.collocate([(), neumann], [(), neumann_bar], initial_obs + extra_initial_obs, op_system, fun_args)
    cur_posterior = initial_posterior
    posteriors = [cur_posterior]
    for t in times[1:]:
        eval_int = augment_with_time(x_pts, t)
        eval_bdy = augment_with_time(x_bdy, t)
        eval_last = augment_with_time(x_pts, t-dt)
        # first compute the observations
        operated_int = cur_posterior.apply_operators([L_explicit], [L_bar_explicit])
        mu, Sigma = operated_int(eval_last)
        #print np.mean(np.diag(Sigma))
        #mu_err, sigma_err = error_dist(x_pts, cur_posterior, t, dt, fun_args)
        mu_err, sigma_err = 0,0
        #print np.mean(mu), np.mean(mu_err)
        obs = [
            (eval_int, mu+mu_err, Sigma+sigma_err),
            (eval_bdy, np.array([[0.], [0.]]))
        ]
        
        cur_posterior = bpdes.collocate(ops, ops_bar, obs, op_system, fun_args)
        posteriors.append(cur_posterior)
    return posteriors

def error_dist(eval_pts, posterior, t_n, dt, fun_args):
    llbar_inv = posterior.__LLbar_inv__
    obs = posterior.__obs__
    g = np.row_stack([o[1] for o in obs])
    p_ops = posterior.__operators__
    p_ops_bar = posterior.__operators_bar__
    
    tdiff = op_system[(k_t,)]
    tbardiff = op_system[(k_tbar,)]
    k = op_system[()]
    
    t_np1 = t_n + dt
    
    eval_t = augment_with_time(eval_pts, t_n)
    eval_tp1 = augment_with_time(eval_pts, t_np1)
    right_dt, left_dt = bpdes.collocation.calc_side_matrices(p_ops, p_ops_bar, obs, eval_t, op_system, [k_t], [k_tbar], fun_args=fun_args)
    right_tp1, left_tp1 = bpdes.collocation.calc_side_matrices(p_ops, p_ops_bar, obs, eval_tp1, op_system, fun_args=fun_args)
    right_t, left_t = bpdes.collocation.calc_side_matrices(p_ops, p_ops_bar, obs, eval_t, op_system, fun_args=fun_args)
    
    left = -left_dt*dt + (left_tp1 - left_t)
    right = -right_dt*dt + (right_tp1 - right_t)
    
    mean_mult = np.dot(left, llbar_inv)
    cov = np.dot(mean_mult, right)
    mean = np.dot(mean_mult, g)
    
    return mean, cov

In [14]:
def plot_parabolic_1d(test_points, posteriors, times, stride=1, deterministic_soln=None):

    means, covs = [], []
    times = times[::stride]
    for p,time in zip(posteriors[::stride], times):
        #print time
        eval_test = augment_with_time(test_points, time)
        mu, cov = p(eval_test)
        means.append(mu)
        covs.append(cov)
    means = np.column_stack(means)
    
    fig = plt.gcf()
    ax = plt.gca()
    line, = ax.plot([], [], c='blue')
    ub, = ax.plot([], [], linestyle='--', c='black')
    lb, = ax.plot([], [], linestyle='--', c='black')
    
    if deterministic_soln is not None:
        deterministic_soln=deterministic_soln[::stride]
        line_det, = ax.plot([], [], c='red')
        
    # initialization function: plot the background of each frame
    def init():
        return []

    # animation function.  This is called sequentially
    def animate(i):
        cov = covs[i]
        mean = means[:,i]
        line.set_data(test_points, mean)
        ub.set_data(test_points, mean+np.diag(cov))
        lb.set_data(test_points, mean-np.diag(cov))
        ax.set_title('t={}'.format(times[i]))
        if deterministic_soln is not None:
            line_det.set_data(points_det, deterministic_soln[i])
        return line,

    # call the animator.  blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=means.shape[1], interval=20, blit=False)
    plt.close()
    return anim

In [15]:
import scipy.sparse, scipy.sparse.linalg
dt_det = 1e-4
alpha_det = 1.0
theta_det = 0.5

points_det = np.linspace(0,Len, 101)
dx = points_det[1] - points_det[0]

times_det = np.arange(0, Tend+dt_det, dt_det)
initial = u0(points_det)
us = [initial]
u_curr = initial

ones = np.ones(points_det.shape[0])
main_mat = scipy.sparse.diags([-(1-theta_det)*alpha_det*ones[:-1]*dt_det/dx**2, 
                               1+2*(1-theta_det)*alpha_det*ones*dt_det/dx**2, 
                               -(1-theta_det)*alpha_det*ones[:-1]*dt_det/dx**2
                              ], [-1,0,1], format='csr')
rhs_mat = scipy.sparse.diags([theta_det*alpha_det*ones[:-1]*dt_det/dx**2, 
                               1 - 2*theta_det*alpha_det*ones*dt_det/dx**2, 
                               theta_det*alpha_det*ones[:-1]*dt_det/dx**2
                              ], [-1,0,1], format='csr')
main_mat[0,0] = -1/dx
main_mat[0,1] = 1/dx
main_mat[-1,-1] = 1/dx
main_mat[-1,-2] = -1/dx
rhs_mat[0,0] = 0
rhs_mat[0,1] = 0
rhs_mat[-1, -1] = 0
rhs_mat[-1, -2] = 0
for t in times_det[1:]:
    rhs = rhs_mat.dot(u_curr)
    u_curr = scipy.sparse.linalg.spsolve(main_mat, rhs)
    us.append(u_curr)

/Users/benorn/.virtualenvs/stats/lib/python2.7/site-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [16]:
main_mat.toarray()

array([[-100. ,  100. ,    0. , ...,    0. ,    0. ,    0. ],
       [  -0.5,    2. ,   -0.5, ...,    0. ,    0. ,    0. ],
       [   0. ,   -0.5,    2. , ...,    0. ,    0. ,    0. ],
       ..., 
       [   0. ,    0. ,    0. , ...,    2. ,   -0.5,    0. ],
       [   0. ,    0. ,    0. , ...,   -0.5,    2. ,   -0.5],
       [   0. ,    0. ,    0. , ...,    0. , -100. ,  100. ]])

In [17]:
def plot_deterministic_1d(test_points, points, times, stride=1):
    points = points[::stride]
    times = times[::stride]
    fig = plt.gcf()
    ax = plt.gca()
    line, = ax.plot([], [])
    # initialization function: plot the background of each frame
    def init():
        return []

    # animation function.  This is called sequentially
    def animate(i):
        p = points[i]
        line.set_data(test_points, p)
        ax.set_title('t={}'.format(times[i]))
        return line,

    # call the animator.  blit=True means only re-draw the parts that have changed.
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=len(points), interval=20, blit=False)
    plt.close()
    return anim

In [18]:
plt.xlim(0, 1)
plt.ylim(0,1)
plot_deterministic_1d(points_det, us, times_det, 100)

In [19]:
dt = 1e-4
times = np.arange(0, Tend, dt)
movie_thin = 10
ext_base = np.concatenate([base_args, [0., 0.]])

In [20]:
def time_design_function(t):
    return augment_with_time(x_pts, t)
def other_obs_function(t):
    bdy_t = augment_with_time(x_bdy, t)
    return [(bdy_t, np.array([[0.], [0.]]))]

In [21]:
#posteriors_euler = solve_theta(ops, ops_bar, ics, times, 1., base_args)
posteriors_euler = bpdes.parabolic.solve_theta([time_op, time_op_bar], 
                                               [[neumann], [neumann_bar]], 
                                               base_op_system,
                                               time_design_function,
                                               other_obs_function,
                                               ics,
                                               times,
                                               1.,
                                               ext_base
                                              )
test_points = np.linspace(0,Len, 51)[:,None]
plt.xlim(x_pts.min(), x_pts.max())
plt.ylim(0., 1.)
plot_parabolic_1d(test_points, posteriors_euler, times, movie_thin, deterministic_soln=us)

In [22]:
#posteriors_backward = solve_theta(ops, ops_bar, ics, times, 0., base_args)
posteriors_backward = bpdes.parabolic.solve_theta([time_op, time_op_bar], 
                                               [[neumann], [neumann_bar]], 
                                               base_op_system,
                                               time_design_function,
                                               other_obs_function,
                                               ics,
                                               times,
                                               0.,
                                               ext_base
                                              )
plt.xlim(x_pts.min(), x_pts.max())
plt.ylim(0., 1.)
plot_parabolic_1d(test_points, posteriors_backward, times, movie_thin, deterministic_soln=us)

In [23]:
posteriors_cn = solve_theta(ops, ops_bar, ics, times, 0.5, base_args)
plt.xlim(x_pts.min(), x_pts.max())
plt.ylim(0., 1.)
plot_parabolic_1d(test_points, posteriors_cn, times, movie_thin, deterministic_soln=us)

[  1.00000000e+00   2.00000000e-02   1.00000000e-01   1.00000000e-04
   5.00000000e-01]


In [24]:
err_mus = []
err_covs = []
for i, p in enumerate(posteriors_euler):
    fun_args = np.append(base_args, [dt, 1.])
    time = times[i]
    err_mu, err_cov = error_dist(x_pts, p, time, dt, fun_args)
    err_mus.append(err_mu)
    err_covs.append(err_cov)
plt.xlim(0, 1)
plt.ylim(np.concatenate(err_mus).min(), np.concatenate(err_mus).max())
plot_deterministic_1d(x_pts, err_mus, times, movie_thin)

Exception: Don't have any operator corresponding to ('implicit', <function k_tbar at 0x112540230>)

In [ ]:
err_mus = []
err_covs = []
for i, p in enumerate(posteriors_backward):
    fun_args = np.append(base_args, [dt, 0.])
    time = times[i]
    err_mu, err_cov = error_dist(x_pts, p, time, dt, fun_args)
    err_mus.append(err_mu)
    err_covs.append(err_cov)
plt.xlim(0, 1)
plt.ylim(np.concatenate(err_mus).min(), np.concatenate(err_mus).max())
plot_deterministic_1d(x_pts, err_mus, times, movie_thin)

In [ ]:

err_mus = []
err_covs = []
for i, p in enumerate(posteriors_cn):
    fun_args = np.append(base_args, [dt, 0.5])
    time = times[i]
    err_mu, err_cov = error_dist(x_pts, p, time, dt, fun_args)
    err_mus.append(err_mu)
    err_covs.append(err_cov)
plt.xlim(0, 1)
plt.ylim(np.concatenate(err_mus).min(), np.concatenate(err_mus).max())
plot_deterministic_1d(x_pts, err_mus, times, movie_thin)